<a href="https://colab.research.google.com/github/cnchandl/Chandler_DSPN_S25/blob/main/ex10_mixed_effects_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/CoAxLab/Data-Explorations/blob/main/book/exercises/mixed-effects-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 10: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models.

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again.

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file.

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

1. Loading and formatting the data 1/1
2. Model fitting 4/4
3. Model assessment 4/4
4. Reflection 1/1

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**.

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [ ]:
# WRITE YOUR CODE HERE
library(tidyverse)


system("gdown --id 1GzKkCaxm8TQN11CXRoyqdg2VOdk3XF4B")
system("gdown --id 1uoNCE8ULzLTkqIxVSoBd0E6GcrwkABUx")

df1 = read.csv("LexicalData.csv") %>%
          mutate(Word = D_Word) %>%
          mutate(D_RT = gsub(",","", D_RT)) %>%
          mutate(D_RT = as.numeric(D_RT))
df2 = read.csv("Items.csv") %>%
         select(Word, Length, Log_Freq_HAL)
df3 = df1 %>%
      left_join(df2) %>%
      drop_na()

Joining with `by = join_by(Word)`


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE
lin.model1 = lm(D_RT ~ Log_Freq_HAL*Length, data = df3)
summary(lin.model1)


Call:
lm(formula = D_RT ~ Log_Freq_HAL * Length, data = df3)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library.

In [ ]:
# WRITE YOUR CODE HERE
#install.packages("lme4")
library(lme4)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rbibutils’, ‘Rdpack’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘RcppEigen’


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output.

In [ ]:
# WRITE YOUR CODE HERE
lin.model2 = lmer(D_RT ~ Log_Freq_HAL*Length + (1|Sub_ID), data = df3)
summary(lin.model2)


Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: df3

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why?

> *fixed effects model: log_freq_hal, t = -3.06; length, t = 29.18; log_freq_hal x length, t = -12.53*

> *mixed effects model: log_freq_hal, t= -4.70; length, t = 36.28; log_freq_hal x length, t = -15.24*

> *the fixed effects models have lower t-values for each of the different terms because all of the variance is attributed to the predictors, whereas the mixed effects model accounts for variability across subjects*

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better?

In [ ]:
# WRITE YOUR CODE HERE
compare = AIC(lin.model1, lin.model2)
compare
diff(compare$AIC)

,df,AIC
,<dbl>,<dbl>
lin.model1,5,914436.4
lin.model2,6,888247.6


[1] -26188.82

> *the mixed effects model has a lower AIC making it a better fit for the data*
>

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set?

> *The trial number may also be important to control for*
>

**DUE:** 11:59 pm EST, March 17, 2025

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*